In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import random

In [4]:
data = pd.read_csv('diabetes.csv')
# Nowy plik na rozmyte od 0

In [5]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
data = data[data['Glucose'] != 0]
data = data[data['BloodPressure'] != 0]
data = data[data['SkinThickness'] != 0]
data = data[data['Insulin'] != 0]
data = data[data['BMI'] != 0]
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
# Klasa statyczna zawierająca metody przydatne to przetwarzania danych
class ProcessingData:

    # metoda tasująca wiersze pd.DataFrame
    @staticmethod
    def shuffle(df: pd.DataFrame) -> pd.DataFrame:
        """
        Receives a DataFrame, randomly shuffles it and returns a shuffled version.
        :param df: pd.DataFrame
        :return df: pd.DataFrame
        """
        for idx in range(len(df) - 1, 0, -1):
            rand_idx = random.randint(0, idx)
            df.iloc[idx], df.iloc[rand_idx] = df.iloc[rand_idx], df.iloc[idx]
        return df.reset_index(drop=True)

    # metoda normalizująca kolumny pd.DataFrame za pomocą normalizacji z użyciem odchylenia standardowego
    @staticmethod
    def normalize_std(df: pd.DataFrame, label: str) -> pd.DataFrame:
        """
        Receives a DataFrame and a class label to skip during normalization, normalizes data using min-max normalization and returns the normalized version.
        :param label: str
        :param df: pd.DataFrame
        :return df: pd.DataFrame
        """
        normalized = df.drop(labels=label, axis=1)
        normalized = (normalized-normalized.mean())/(normalized.std())
        return normalized.join(df[label])

    # metoda normalizująca kolumny pd.DataFrame za pomocą normalizacji minmax
    @staticmethod
    def normalize_minmax(df: pd.DataFrame, label: str) -> pd.DataFrame:
        """
        Receives a DataFrame and a class label to skip during normalization, normalizes data using min-max normalization and returns the normalized version.
        :param label: str
        :param df: pd.DataFrame
        :return df: pd.DataFrame
        """
        normalized = df.drop(labels=label, axis=1)
        normalized = (normalized-normalized.min())/(normalized.max()-normalized.min())
        return normalized.join(df[label])

    # metoda dzieląca pd.DataFrame na zbiór treningowy oraz walidacyjny
    @staticmethod
    def split(df: pd.DataFrame, ratio: float) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        Receives a DataFrame and returns two dataframes, split into training and validation DataFrames.
        :param df: pd.DataFrame
        :param ratio: float
        :return training_df, validation_df: tuple[pd.DataFrame, pd.DataFrame]
        """
        cutoff = int(len(df) * ratio)
        return df[:cutoff].reset_index(drop=True), df[cutoff:].reset_index(drop=True)

In [8]:
shuffled_data = ProcessingData.shuffle(data)
training_data, _ = ProcessingData.split(shuffled_data, 0.7)

In [16]:
class Fuzzy:
    def __init__(self):
        # dictionary of dictionaries of antecedents
        self.antecedents = {}
        # TODO: decide what this is
        self.consequent = None
        # TODO: choose the implementation
        self.rules = []

    @staticmethod
    def triangular_function(n: float, a: float, b: float, c: float) -> float:
        # print(n, a, b, c)
        if n <= a:
            return 0
        if a < n < b:
            return (n - a) / (b - a)
        if n == b:
            return 1
        if b < n < c:
            return (c - n) / (c - b)
        if n >= c:
            return 0

    @staticmethod
    def trapezoidal_function(n: float, a: float, b: float, c: float, d: float) -> float:
        # print(n, a, b, c, d)
        if c == d and n >= d:
            return 1
        if a == b and n <= a:
            return 1
        if n <= a:
            return 0
        if a < n < b:
            return (n - a) / (b - a)
        if b <= n <= c:
            return 1
        if c < n < d:
            return (d - n) / (d - c)
        if n >= d:
            return 0

    def membership_function(self, n: float, *args) -> None:
        # print(len(args))
        # print(*args)
        if len(args) == 3:
            print(self.triangular_function(n, *args))
        elif len(args) == 4:
            print(self.trapezoidal_function(n, *args))
        else:
            print("Nieprawidłowa liczba argumentów")

    def add_antecedent(self, parameter: str, linguistic_value: str, *args) -> None:
        # TODO: change the implementation
        self.antecedents.append([parameter, linguistic_value, *args])

    def add_consequent(self, parameter: str, linguistic_value: str, *args) -> None:
        # TODO: potentially change the implementation
        self.antecedents.append([parameter, linguistic_value, *args])

    def add_rule(self):
        # TODO: implementation
        print(self.rules)

    @staticmethod
    def view(self, parameter: str, linguistic_value: str) -> None:
        # TODO: plotting an antecedent
        print()
        # x = np.linspace(0., 7., num=101)
        # y = np.array([fuzzy.membership_function(i, 1, 1, 3, 7) for i in x])
        #
        # fig, ax = plt.subplots()
        # ax.plot(x, y)
        # loc = plticker.MultipleLocator(base=1.0)
        # ax.xaxis.set_major_locator(loc)
        #
        # plt.plot(x, y, 'r')
        # plt.show()

In [21]:
fuzzy = Fuzzy()

In [28]:
# fuzzy.membership_function(9, 5, 10, 17, 17) # high
# print()
# fuzzy.membership_function(145, 125, 155, 199, 199) # high
# print()
# fuzzy.membership_function(80, 90, 102, 122, 122) # high
# print()
# fuzzy.membership_function(46, 36, 43, 99, 99) # high
# print()
# fuzzy.membership_function(130, 76, 125, 190) # medium
# fuzzy.membership_function(130, 125, 190, 846, 846) # high
# print()
# fuzzy.membership_function(37.9, 28, 37, 67.1, 67.1) # high
# print()
# fuzzy.membership_function(0.637, 0.25, 0.5, 0.75) # medium
# fuzzy.membership_function(0.637, 0.5, 0.75, 2.42, 2.42) # high
# print()
# fuzzy.membership_function(40, 21, 21, 34, 45) # low
# fuzzy.membership_function(40, 34, 45, 56) # medium